In [ ]:
"""
Created on Wed Sep 22 15:27 2021

This is a script to create subplots of different regions (in 2D)

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import cartopy
import cartopy.crs as ccrs
from cartopy.util import add_cyclic_point
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
from matplotlib.ticker import LogFormatterSciNotation
import matplotlib.colors as colors

import basal_melt_param.map_plot_functions as mapfunc

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

READ IN DATA

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

In [ ]:
nemo_run = 'OPM027'

In [ ]:
inputpath_mask = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/ANTARCTICA_IS_MASKS/nemo_5km_'+nemo_run+'/'
outputpath_melt = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/processed/MELT_RATE/nemo_5km_'+nemo_run+'/'
plot_path = '/bettik/burgardc/PLOTS/NN_plots/'
inputpath_boxes = '/bettik/burgardc/SCRIPTS/basal_melt_param/data/interim/BOXES/nemo_5km_'+nemo_run+'/'
outputpath_melt_nn = '/bettik/burgardc/DATA/NN_PARAM/processed/MELT_RATE/'

In [ ]:
timetag = '20220414-1706'
new_path_output = outputpath_melt_nn+timetag+'/'

In [ ]:
melt_rate_2D_NEMO = xr.open_dataset(outputpath_melt+'melt_rates_2D_NEMO_timmean.nc')
box_charac_all_2D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_2D.nc')
box_charac_all_1D = xr.open_dataset(inputpath_boxes + 'nemo_5km_boxes_1D.nc')

In [ ]:
melt_rate_2D_nn = xr.open_dataset(new_path_output+'NN_melt_predicted_reference_m_ice_per_yr_'+nemo_run+'.nc')
melt_rate_2D_nn_timmean = melt_rate_2D_nn['predicted_melt'].mean('time').reindex_like(file_isf['ISF_mask'])

In [ ]:
file_isf_orig = xr.open_dataset(inputpath_mask+'nemo_5km_isf_masks_and_info_and_distance_new.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
large_isf = file_isf_nonnan['Nisf'].where(file_isf_nonnan['isf_area_here'] >= 2500, drop=True)
file_isf = file_isf_nonnan.sel(Nisf=large_isf)
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
file_isf_orig

In [ ]:
ref_melt_2D = melt_rate_2D_NEMO['melt_m_ice_per_y'].where(file_isf_mask)

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
box_loc_config2 = box_charac_all_2D['box_location'].sel(box_nb_tot=box_charac_all_1D['nD_config'].sel(config=2))
box1 = box_loc_config2.where(box_loc_config2==1).isel(Nisf=0).drop('Nisf')
box1_msk = box1.where(box1==1,0)

PLOT WHOLE ANTARCTICA

In [ ]:
f = mapfunc.map_with_contourf_coolwarm(ref_melt_2D, grounded_msk, icesheet_msk, 'Reference (NEMO)')
#f.savefig(plot_path+'spatial_NEMO_ref_'+nemo_run+'.png',dpi=300)

In [ ]:
f = mapfunc.map_with_contourf_coolwarm(melt_rate_2D_nn_timmean, grounded_msk, icesheet_msk, 'NN '+timetag)

SINGLE ICE SHELVES

In [ ]:
kisf = 10
if kisf == 11:
    kisf_mask = (file_isf_mask==11) | (file_isf_mask==21)
else:
    kisf_mask = file_isf_mask==kisf

ref_melt_isf = ref_melt_2D.where(kisf_mask,drop=True)
print('x size',ref_melt_isf.x.max() - ref_melt_isf.x.min())
print('y size',ref_melt_isf.y.max() - ref_melt_isf.y.min())


In [ ]:
kisf = 48
if kisf == 11:
    kisf_mask = (file_isf_mask==11) | (file_isf_mask==21)
else:
    kisf_mask = file_isf_mask==kisf

ref_melt_isf = ref_melt_2D.where(kisf_mask,drop=True)
print('y max =',ref_melt_isf.y.max())
print('x min =',ref_melt_isf.x.min())

try other colorbar

In [ ]:
def myround(x, base=5):
    return (base * np.ceil(x/base)).astype(int)

In [ ]:
myround(11.5)

In [ ]:
dom = 50

kisf = 66
if kisf == 11:
    kisf_mask = (file_isf_mask==11) | (file_isf_mask==21)
else:
    kisf_mask = file_isf_mask==kisf

ref_melt_isf = ref_melt_2D.where(kisf_mask,drop=True)
cmaph = mpl.cm.coolwarm
    
if kisf == 11:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 10000
elif kisf == 10:
    ymax = ref_melt_isf.y.min() - 80000
    xmin = ref_melt_isf.x.min() + 5000
elif kisf == 31:
    ymax = ref_melt_isf.y.min() - 20000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 44:
    ymax = ref_melt_isf.y.min() - 30000
    xmin = ref_melt_isf.x.min() + 3000 
elif kisf == 48:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 3000   
elif kisf == 66:
    ymax = ref_melt_isf.y.min() - 15000
    xmin = ref_melt_isf.x.min() + 1000   
    
    
param_melt_isf = melt_rate_2D_nn_timmean.where(kisf_mask,drop=True)

if (kisf == 11) or (kisf==10):
    min_maxval = 5
elif kisf == 44:
    min_maxval = 5
elif kisf == 31:
    min_maxval = 10
else:
    min_maxval = myround(abs(ref_melt_isf).max().values)

levs_neg = np.linspace(-min_maxval,0,min_maxval+1)
levs_pos = np.linspace(0,min_maxval,min_maxval+1)
levs = np.concatenate([levs_neg,levs_pos[1::]])
round_lev = np.round(levs,0)


max_list = []

fig, ax = plt.subplots(1,3,sharex=True,sharey=True)
fig.set_size_inches(8.25, 8.25/2.5)
for i in range(len(ax)):
    if i==0:
        cbar_ax2 = ref_melt_isf.plot.contourf(ax=ax[i],cmap=cmaph, zorder=0, levels=levs, vmin=-min_maxval, vmax=min_maxval, add_colorbar=False)  #levels=levs
        ax[i].text(xmin,ymax,'Max melt = '+str(np.round(ref_melt_isf.max().values,1)))
    elif i==1:
        param_melt_isf.plot.contourf(ax=ax[i],cmap=cmaph, levels=levs, zorder=0, add_colorbar=False)  
        max_quad_Ant = param_melt_isf.max().values
        max_list.append(max_quad_Ant)
        ax[i].text(xmin,ymax,'Max melt = '+str(np.round(max_quad_Ant,1)))
        
        #if np.max(max_list) > ref_melt_isf.max().values:
        #    cbar = fig.colorbar(cbar_ax2, extend='max',orientation='horizontal')
        #else:
        #    cbar = fig.colorbar(cbar_ax2,orientation='horizontal')
        
        
    elif i==2:
        (param_melt_isf - ref_melt_isf).plot.contourf(ax=ax[i],cmap=cmaph, zorder=0, add_colorbar=True, vmin=-10, vmax=10)  
        
    ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,grounded_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=10)
    ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,box1_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='blue',zorder=10)
    ax[i].set_xticks([])
    ax[i].set_yticks([])
    ax[i].set_xlabel('')
    ax[i].set_ylabel('')
    ax[i].set_title('')
      
    #ax[i].contour(ref_melt_isf.x,ref_melt_isf.y,icesheet_msk.where(ref_melt_isf),levels=[0,1],linewidths=0.5,colors='black',zorder=15)

        
if kisf in [48]:
    cbar = fig.colorbar(cbar_ax2, ticks=round_lev[::3],orientation='horizontal')
elif kisf in [10, 11]:
    cbar = fig.colorbar(cbar_ax2, ticks=round_lev[::2],orientation='horizontal')
elif kisf in [66]:
    cbar = fig.colorbar(cbar_ax2, ticks=round_lev[::20],orientation='horizontal')
elif kisf in [31]:
    cbar = fig.colorbar(cbar_ax2, ticks=round_lev[::5],orientation='horizontal')
elif kisf in [44, 31]:
    cbar = fig.colorbar(cbar_ax2, ticks=round_lev[::2],orientation='horizontal')
else:
    cbar = fig.colorbar(cbar_ax2,orientation='horizontal')

plt.tight_layout()
fig.savefig(plot_path+'comparison_spatial_patterns_isf'+str(kisf).zfill(3)+'_'+str(nemo_run)+'_'+str(timetag)+'_with_colorbar.png', dpi=300)

In [ ]:
ref_melt_isf = ref_melt_2D.where(kisf_mask,drop=True)
cmaph = mpl.cm.coolwarm
norm = colors.TwoSlopeNorm(vmin=-10, vcenter=0, vmax=70)
ref_melt_isf.plot.contourf(cmap=cmaph,norm=norm)

In [ ]:
np.round(levs,0).astype(int)

In [ ]:
np.concatenate([levs_neg,levs_pos[1::]])

In [ ]:
melt_rate_2D_boxes['melt_m_ice_per_y'].sel(param='boxes_2_pismno_picopno_tuned_clara', profile_domain=50).plot()

In [ ]:
np.round(grounded_msk.where(ref_melt_isf)).plot()

In [ ]:
icesheet_msk.where(file_isf_mask==66,drop=True).plot()

In [ ]:
plt.contour(ref_melt_isf.x,ref_melt_isf.y,grounded_msk.where(ref_melt_isf),levels=[-1,2])

In [ ]:
grounded_msk.min()

In [ ]:
min_maxval